In [1]:
2+2

4

In [2]:
import json
import os
from dotenv import load_dotenv
# Load environment variables from .env
load_dotenv("/home/jovyan/.envrc")

True

In [3]:
import os
import json
import random  # Import random for shuffling

# Directory containing the data files
data_dir = os.path.abspath('../reviews-assistant/data/reviews')

# Initialize an empty list to hold all reviews
reviews = []

# List objects in the directory
objects_in_directory = os.listdir(data_dir)

# Iterate over the files in the directory
for obj in objects_in_directory:
    if obj.endswith('.json'):  # Check if the file is a JSON file
        file_path = os.path.join(data_dir, obj)
        with open(file_path, 'r', encoding='utf-8') as jsonfile:
            # Load the reviews from the JSON file
            file_reviews = json.load(jsonfile)
            
            # If the loaded content is a list of reviews, extend the main list
            if isinstance(file_reviews, list):
                reviews.extend(file_reviews)  # Append reviews to the main list
            else:
                reviews.append(file_reviews)  # Handle the case where it's not a list

# Shuffle the reviews to randomize their order
random.shuffle(reviews)

# Print the first i shuffled reviews
i = 2  # Change this to print more reviews if needed
for review in reviews[:i]:
    print(f"Author ID: {review['author.steamid']}")
    print(f"Review: {review.get('review', 'No text')}")
    print(f"Timestamp Created: {review['timestamp_created']}")
    print("-" * 79)


Author ID: 76561198072094217
Review: After putting in the same amount of hours that i needed to 100% finish Odyssey here I am finishing 100% of Valhalla. It was a nice adventure with a lot of fun and cool quests and locations.

[h1] The good [/h1]

They really nailed the viking experience I felt watching the Vikings tv show from 2013. Even tho it felt like a mini game the whole raiding settlements and monasteries I did enjoy it a lot, but kinda wished they did more with it.

The story was ok and I really liked what they did with the whole reincarnation of the 'old gods'. The story could have been better presented or at least give some weight to the alliances you get over the time of playing. After finishing a zone and you get that alliances you might never get a chance to ever go back and you will only see 1 or 2 characters from that zone in conquests to just kick the bucket or tell you they won't help you in the future. This was missed potential and I wished they implemented the story

In [4]:
len(reviews)

1782

In [5]:
reviews[-1]

{'appid': '2208920',
 'timestamp_query': 1727643597,
 'title': "Assassin's Creed Valhalla",
 'recommendationid': '174722275',
 'author.steamid': '76561199291372999',
 'author.playtimeforever': None,
 'author.playtime_last_two_weeks': 0,
 'author.playtime_at_review': 8996,
 'author.last_played': 1726115935,
 'language': 'english',
 'review': 'The Graphics are amazing. I also enjoyed the story line. I never expected Eivor to be the leader eventually. I thought he was just the caretaker.',
 'voted_up': True,
 'votes_up': 0,
 'timestamp_created': 1726050285,
 'timestamp_updated': 1726050285}

In [6]:
from openai import OpenAI

client = OpenAI()

In [7]:
prompt_template = """
You are a PC video game enthusiast who enjoys playing games on their release day.
You are aware that it is now common for developers to release unfinished or unoptimized products.
You understand that developers often fail to deliver on the promises made about a game prior to its release.
You prefer games that function well upon launch and are delivered with proper quality—titles that are not released prematurely.
Also, a user does not want to use any external applications for logging or sharing data. They want only to use Steam as a means of buying and playing video games.

Formulate 5 questions this user would ask based on the provided inputed review from Steam store.
Create only questions that are related to this exercise and type of user.
The record should contain the answer to the questions, and the questions should be complete and not too short. Use as few words as possible from the record.
That will be the answer field of our output JSON file.

After that, assign a single-word label to each question and store it as a label.
That will be the section field of our output JSON file.
The values for section can cover any issue a video gamer might be interested in: audio, video, hardware, plot, characters, ame mechanics, multiplayer opponents' toxicity, game-breaking bugs,, enemy variety, location variety, violence level, DEI (Diversity, Equity, Inclusion)presence, whether the game is complete, whether the game is functional, whether it is worth the price, etc. Be creative here.
Also in output JSON file, assign to each question-answer a review which was used for text generation.

The input record:

appid: {appid}
timestamp_query: {timestamp_query}
title: {title}
recommendationid: {recommendationid}
language: {language}
review: {review}
voted_up: {voted_up}
votes_up: {votes_up}
timestamp_created: {timestamp_created}
timestamp_updated: {timestamp_updated}

Provide the output in parsable JSON without using code blocks:

{{"review": ["review1"], "question": ["question1", "question2", ..., "question5"], "answer": ["answer1", "answer2", ..., "answer5"], "section": ["section1", "section2", ..., "section5"]}}
""".strip()

In [8]:
prompt = prompt_template.format(**reviews[0])

In [9]:
prompt

'You are a PC video game enthusiast who enjoys playing games on their release day.\nYou are aware that it is now common for developers to release unfinished or unoptimized products.\nYou understand that developers often fail to deliver on the promises made about a game prior to its release.\nYou prefer games that function well upon launch and are delivered with proper quality—titles that are not released prematurely.\nAlso, a user does not want to use any external applications for logging or sharing data. They want only to use Steam as a means of buying and playing video games.\n\nFormulate 5 questions this user would ask based on the provided inputed review from Steam store.\nCreate only questions that are related to this exercise and type of user.\nThe record should contain the answer to the questions, and the questions should be complete and not too short. Use as few words as possible from the record.\nThat will be the answer field of our output JSON file.\n\nAfter that, assign a si

In [10]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [11]:
questions = llm(prompt)

In [12]:
json.loads(questions)

{'review': ['After putting in the same amount of hours that i needed to 100% finish Odyssey here I am finishing 100% of Valhalla. It was a nice adventure with a lot of fun and cool quests and locations.'],
 'question': ['Does the game feel complete at launch, or are there unfinished elements that detract from the experience?',
  'How well does the game perform without issues or bugs on release day?',
  'Is there a good variety of quests and gameplay elements that make the game engaging?',
  'Does the storyline feel cohesive and satisfying throughout the gameplay experience?',
  'Is the game worth the full price considering the quality of content provided at launch?'],
 'answer': ['The game feels somewhat incomplete, especially with the implementation of the story and DLCs.',
  'There are concerns about combat being repetitive and some segments being poorly received.',
  'The quests are fun, but some aspects felt scattered and lacking depth.',
  'The storyline has missed potential and f

In [13]:
import json
from tqdm.auto import tqdm

def generate_questions(doc):
    prompt = prompt_template.format(**doc)

    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )

    json_response = response.choices[0].message.content
    return json.loads(json_response)

final_results = []

# Initialize a unique ID counter starting from 0
unique_id_counter = 0

# Single pass: generating questions and structuring final results
for doc in tqdm(reviews):
    doc_id = doc['appid']

    try:
        # Generate questions, answers, and sections
        questions_data = generate_questions(doc)

        # Loop through questions, answers, and sections and add to final results
        questions = questions_data['question']
        answers = questions_data['answer']
        sections = questions_data['section']

        # Add to final results using the current document
        for i in range(len(questions)):
            final_results.append({
                "document_id": unique_id_counter,  # Unique ID starting from 0 and incrementing
                "appid": doc_id,                 # Add the appid for the game
                "review": doc,                   # Add the current review document
                "question": questions[i],         # Add the question
                "answer": answers[i],             # Add the corresponding answer
                "section": sections[i]            # Add the corresponding section
            })

            unique_id_counter += 1  # Increment the unique ID counter for the next entry

    except json.JSONDecodeError as e:
        print(f"Error processing document with appid {doc_id}: JSONDecodeError - {e}")
    except Exception as e:
        print(f"Error processing document with appid {doc_id}: {e}")

  0%|          | 0/1782 [00:00<?, ?it/s]

Error processing document with appid 315210: JSONDecodeError - Expecting ',' delimiter: line 1 column 1036 (char 1035)
Error processing document with appid 721180: JSONDecodeError - Expecting ',' delimiter: line 1 column 4322 (char 4321)
Error processing document with appid 2698940: JSONDecodeError - Expecting ',' delimiter: line 1 column 1526 (char 1525)
Error processing document with appid 721180: JSONDecodeError - Expecting ',' delimiter: line 1 column 687 (char 686)
Error processing document with appid 2443720: JSONDecodeError - Expecting ',' delimiter: line 1 column 749 (char 748)
Error processing document with appid 2322010: JSONDecodeError - Unterminated string starting at: line 1 column 2100 (char 2099)
Error processing document with appid 2369390: JSONDecodeError - Expecting ',' delimiter: line 1 column 771 (char 770)
Error processing document with appid 582160: JSONDecodeError - Expecting ',' delimiter: line 1 column 27 (char 26)
Error processing document with appid 315210: J

In [15]:
import json
import os

# Assuming final_results is defined as a list of dictionaries
# Example: final_results = [{'recommendationid': 1, 'question': 'What is the best game?'}, ...]
data_dir = os.path.abspath('../reviews-assistant/data/ground_truth')

# Define the output file path
output_file = os.path.join(data_dir, "ground_truth_retrieval.json")

# Save the results to a JSON file
with open(output_file, 'w', encoding='utf-8') as json_file:
    json.dump(final_results, json_file, ensure_ascii=False, indent=4)

print(f"Data saved to {output_file}")


Data saved to /home/jovyan/reviews-assistant/data/ground_truth/ground_truth_retrieval.json
